Modified: Jul 31, 2019

# Evolution of curves using SDF

- Define a step function that encodes the discrete PDF of the curve evolution 
- Visualize the contour lines (at zero level)
- Active Contour 
    - satellite images
    - biomedical images
- Next step: 
    - agent-based clustering for image segmentation


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp
p = print

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

## Import helper functions

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

# pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint, timeit
import utils

import sdfs 
from vector import Vector as vec
from samples import LSTestSample

In [ ]:
import calculus as calc
from grid import CartesianGrid

## Set visualization options

In [ ]:
H, W = 500,500

In [ ]:
opts.defaults(
    opts.Image(colorbar=True, active_tools=['wheel_zoom'], tools=['hover']),
    opts.Curve(tools=['hover'], active_tools=['wheel_zoom']),
    opts.RGB(active_tools=['wheel_zoom'], tools=['hover'])
)

In [ ]:
img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=False, 
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

---
## Curve Evolution 

Todo
- [ ] Finite Difference method on parametric equations
- [ ] Levelset functions
    - 2D signed distance functions: [src](https://is.gd/t7p5mk)
- [ ] Active contour on satellite images
- [ ] Agent-based modelling with specified rules
    - satellite image segmentation (~ clustering based on local features)

### Front Propagation 
Refer to Equation 4.8 and 4.20

1. Spatial gradient computation </br>
For stability in computing the spatial gradients, use Eqn. 4.33

2. Temporal discretization </br>
To propagate the front over time, we neet to update the levelset values over time 
according to a process. We express the process as a partial differential of the levelset function $\phi$ wrt time:

$$
\frac{\partial \phi}{\partial t} (t)= L(t, \phi(t)), ~~~~~ \phi(t_0) = \phi^{0}
$$

where $L$ represents a general function that depends on time stamp $t$ and the levelset function itself.
Given $L$ and the initial condition $\phi_{0}$, our goal is to find $\phi(t_{1}), \phi(t_{2})$, ... $\phi(t_{n})$ This process is called **"time integration''**, as we are solving the partial differential equation wrt time.

Using the first-order Taylor expansion around $t + \Delta t$, we derive a "forward" Euler time integration equation for discretely sampled levelset:

$$
\begin{align}
\phi(t+\Delta t) &\approx \phi(t) + \Delta t \frac{\partial \phi}{\partial t}(t) \\
                 &\approx \phi(t) + \Delta t L(t, \phi(t))
\end{align}
$$


The equations we have described both in a continous and discretized domain for $\phi(t)$ hold for any $\phi$, and is not specificed to a levelset function.  In that respect, a better notation to express this generality, I should have used a symbol that is not $\phi$, oops. But, we are interested in the levelset functions (in particular a discretly sampled one), so from here on, we will view this general equation from the perspective of time integration for a levelset equation.  That means, $\phi$ refers to the levelset function, and $L$, which describes how the levelset fucntion changes over time, correpsonds to the speed function $F$ in the fundamental levelset form. Recall this speed function is the speed in direction normal to the levelset function (at all levels).


### Two ways to view the fundamental levelset formulation
1. Advection: floating surface in an external vectorfield/flow $\vec{V}$


2. Motion of a front in normal direction according to a given speed $F$ </br>
The front propagates in its normal direction with a given speed $F(t, \phi)$

The two views are mathmatically equivalent as can be shown by using the equality that links the external vector field point of view to the levelset domain by linking the way to compute normal vector in both domains:

$$
\vec{n}(\vec{x}) = \frac{\vec{\nabla}\phi (\vec{x})}{\lVert \vec{\nabla}\phi (\vec{x}) \rVert}
$$

where $\vec{x}$ refers to a point in the embedding space. In 2Dim case (ie. for curve evolution), we can express this point with two Cartian coordinates $(x,y)$:

$$
\vec{n}(x,y) = \frac{\vec{\nabla}\phi (x,y)}{\lVert \vec{\nabla}\phi (x,y) \rVert}
$$

The key point of this equality is that it proves the two ways to view the fundamental levelset form are mathmatically equivalent. Nevertheless, their physical meanings are distinct, and we solve the equations differently when it comes to finding the computational solutions, ie. $\phi(t_{1}),\phi(t_{2}), \dots$.

The solution to the first approach (ie. advection) will be implemented in `LSEvolver.advect` method. 
The solution to the second approach (ie. front's normal motion) will be implemented in `LSEvolver. propagate` method. 

Before moving on to the impelemntation, let me express the time integration of a discrete levelset function for each perspective

### continue here!! ###
11:45am, Jul 31, 2019
- plug in the V or F function to the time integration (discrete) - the one with \approx -- 
###

1. Advection


2. Front's normal motion




Now we are ready to look at the implementation!

### Implementation

In [ ]:
################################################################################
class LSEvolver(CartesianGrid):
    """
    EvolvingLS: A levelSet evolution according to an initial-valued problem given by a PDE
    
    Args:
    - F (callable): takes a LevelSet object and time index and returns a np array 
    with the same shape as the levelset's grid
    """
    def __init__(self, xs, ys, data=None, t=0):
        super().__init__(xs, ys, data)
        
        self.time = t #current time
        self.delta = np.inf # average change of LS function values between consecutive time stamps
            
    def run(self, F, dt, pde_class, threshold=1e-3, maxIter=1e4):
        count = 0
        deltas = []
        phis = {}
        while self.delta > threshold:
            if count > maxIter: 
                print("MaxIter reached: ", count)
                break
            self.propagate(F,dt,pde_class)
            deltas.append(self.delta)
            count += 1
            if count%1000:
                phis[self.time] = self.data
        print(f"Ran for {count} steps, for {self.time} periods")
        print(f"\taverage delta phi: {self.delta}")
        return deltas, phis

    def propagate(self, F, dt, pde_class):
        """
        Equation 4.8 and 4.20
        1. Spatial gradient computation
        For stability in computing the spatial gradients, use Eqn. 4.33
        
        2. Temporal discretization
        To propagate the front over time, we neet to update the levelset values over time 
        according to a process. We express the process as a partial differential of the levelset 
        function (\phi) wrt time:
        
        \frac
        
        Our goal is to find \phi(t1), \phi(t2), ... given a initial \phi(t0)
        equatation
        This process is called "time integration" 
        
        Args:
        - pde_class (str): 'hyperbolic', 'parabolic'
            * (1) if F depends on at most order 1 derivatives of the levelset function phi 
            wrt space and time, the information propagation has a specific direction 
            (ie. "characteristics"), and we need to be careful about which gradient to 
            take -- backward, forward.  In this case, the levelset equation is 'hyperbolic', 
            which is a subclass of Hamilton-Jacobian equation. 
            
            * (2) if F depends on derivatives of order >= 2 (eg. F = alpha*curvature),
            then the information propagates from all directions, and we can use the 
            central finite difference method to compute the spatial gradients.
        """
        if dt > min(self.dx, self.dy):
            #todo: print error but then make dt smaller smartly
            raise ValueError('dt should be smaller than x and y sample resolutions: ', dt)
                
        assert pde_class in ['hyperbolic','parabolic'], \
        f"pde_class must be either 1 or 2: {pde_class}"
        
        if pde_class == 'hyperbolic':
            dxb, dxf, dyb, dyf = self.get_diff1_bf()
            debug = (
                hv.Image(self.data, label='phi') + hv.Image([])
                + hv.Image(dxb, label='dx back') + hv.Image(dxf, label='dx forward')
                + hv.Image(dyb, label='dy back') + hv.Image(dyf, label='dy forward')
            ).cols(2)
            display(debug)

            S = np.sign(F)
            dx = np.maximum(S*dxb, -S*dxf)
            dy = np.maximum(S*dyb, -S*dyf)

            dmag = np.sqrt(dx**2 + dy**2)
            
        else :#pde_class == 'parabolic':
            dx,dy = self.get_diff1_central()
            dmag = comput_mag(dx,dy)# todo
        
        
        # update phi
        dphi = dt* dmag * F
        self.delta = dphi.sum() / dphi.size
        self.data -= dphi 
        
        # update time
        self.time += dt
        
    def advect(self, V, dt):
        """
        Args:
        - V (ndarray of shape (w,h,2)): containing x and y component of the vector field
        - dt (float): time step size
        """
        pass
    
    def reinit(self, method='sweep'):
        """
        Reset current phi values (in self.data) to satisfy Eikonal equality
        in Eqn. 4.12
        
        - method 
            - 'pde': solve eqn. 4.37 with current phi data, until steady state
            - 'fmm': fast marching method
            - 'sweep' (default): paper [88]
            - 'exact': paper [64]
            
            Default is 'sweep'
        """
        pass

    def get_diff1_bf(self, switch=True):
        dxb, dxf, dyb, dyf = calc.diff1_bf(self.data, switch)
        return dxb/self.dx, dxf/self.dx, dyb/self.dy, dyf/self.dy
    
    def get_diff1_central(self):
        dx, dy= calc.diff1_central(self.data)
        return dx/(2*self.dx), dy/(2*self.dy)
    
    def get_curvature(self):
        return curvature(self.data)
    


## Test LSEvolver